# Hill-climbing

In [ ]:
name_list = [
             '0712-b-stratified-large',
             '0713-a-stratified-large',
             '0718-a-stratified', 
             '0718-b-stratified', 
             '0718-c-stratified', 
             '0718-d-stratified', 
             '0724-a-stratified',
             '0726-a-stratified-large',
             '0726-b-stratified',
             '0726-c-stratified-large',
             '0727-b-stratified-large',
             '0727-c-stratified-large',
             '0727-f-stratified-large',
             '0728-c-stratified',
             '0728-d-stratified',
             '0728-e-stratified',
            '0729-a-stratified-large',
            '0729-b-stratified-large',
            '0729-d-stratified-large',
            '0730-b-stratified',
            '0730-c-stratified-large',
            '0731-a-stratified-large',
            '0731-b-stratified-large',
            '0731-e-stratified-large',
            '0801-c-stratified',
            ]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
pd.options.display.max_colwidth = None
oof_path_list = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    print('---------------')
    for filename in filenames:
        path = os.path.join(dirname, filename)
        print(path)
        if 'oof' in path:
            oof_path_list.append(path)
oof_path_list.sort()

In [ ]:
oof_path_list

In [ ]:
path_df = pd.DataFrame(oof_path_list, columns=['oof_path']).reset_index()

In [ ]:
path_df

In [ ]:
cat_file_list = []
for name in name_list:
    oof_cat = pd.DataFrame([])
    for path in path_df['oof_path']:
        if name in path:
            oof_i_df = pd.read_csv(path)
            oof_cat = pd.concat([oof_cat, oof_i_df],ignore_index=True)
    oof_cat.to_csv(f'cat_{name}.csv')
    print(f'cat_{name}.csv')
    cat_file_list.append(oof_cat)

        

In [ ]:
def cal_rmse(pred_values, target_values):
# np.sqrt(nn.MSELoss(reduction="mean")(torch.tensor(a.pred.values), torch.tensor(a.target.values)).item())
    return np.sqrt(nn.MSELoss(reduction="mean")(torch.tensor(pred_values), torch.tensor(target_values)).item())

In [ ]:
for file in cat_file_list:
    print(cal_rmse(file.pred.values, file.target.values))

In [ ]:
# oof_path_list
# path_df
# name_list
# cat_file_list

In [ ]:
def init_hillclimb():
    best_ensemble = []
    best_score = 1.5
    
    return best_ensemble, best_score

In [ ]:
def find_best_improvement(ensemble, cat_file_list):
    best_score    = 1.5
    best_ensemble = [] 
    # 寻找到label对应的能带来最大均值提升的那个文件
    for i in range(0,len(cat_file_list)):
        ensemble = ensemble + [i] 
        score = score_ensemble(ensemble, cat_file_list) # 加入均值集成，计算分数
        
        if score < best_score:
            best_score    = score
            best_ensemble = ensemble
            
        ensemble = ensemble[:-1]
    
    return best_ensemble, best_score

In [ ]:
def score_ensemble(ensemble, cat_file_list):
    blend_preds = np.zeros(cat_file_list[0].shape[0]) 
    for model_idx in ensemble:
        blend_preds += cat_file_list[model_idx]['pred']
        
    blend_preds = blend_preds/len(ensemble)
    score = cal_rmse(blend_preds, cat_file_list[0]['target'].values)
    
    return score

In [ ]:
def climb(best_ensemble, best_score):
    best_ensemble, best_score = find_best_improvement(best_ensemble, cat_file_list)
    return best_ensemble, best_score

In [ ]:
print(f'mean stacking: {score_ensemble(list(range(len(cat_file_list))), cat_file_list)}')

In [ ]:
def get_optimal_weights(best_ensemble):
    weights = {}
    # 获取最优权重
    for num in set(best_ensemble):
        weights[num] = best_ensemble.count(num)/len(best_ensemble)
    return weights

In [ ]:
best_ensemble, best_score = init_hillclimb()

# Run hillclimb
for i in range(100):
    print("-------------")
    print("Step", i)    
    best_ensemble, best_score = climb(best_ensemble, best_score)
    print("Best ensemble:")
    print(best_ensemble)
    print("Best score:")
    print(best_score)

# Get optimal weights
opt_w = get_optimal_weights(best_ensemble)
print("Optimal weights:")
print(opt_w)

In [ ]:
for i in range((len(cat_file_list))):
    if i not in opt_w:
        opt_w[i] = 0

In [ ]:
opt_w

# Import

In [ ]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup
from transformers import logging
logging.set_verbosity_error()
from sklearn.model_selection import KFold, StratifiedKFold

import gc
gc.enable()

In [ ]:
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL
# example: print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")

In [ ]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

In [ ]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        
            # 注意这里要把 pred移到cpu上
            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

In [ ]:
NUM_FOLDS = 5
NUM_EPOCHS = 6
SEED = 21
BATCH_SIZE = 2
MAX_LEN = 248
NUM_WORKERS = 0
IS_TARGET_SAMPLING = False
IS_FREEZE_EMBEDDING = False
IS_BERT_ADAM = True
IS_STRATIFIED_KFOLD = True
IS_USING_AMP = True
ROBERTA_HIDDEN_LEN = 768 # 768 or 1024
RANDOM_FACTOR_OF_TARGET_SAMPLING = 0.1
EARLY_STOPPING_PATIENCE = 15
EARLY_STOPPING_LOSS_THRES = 0.485
WEIGHT_DECAY = 0.01

EVAL_SCHEDULE = [(0.50, 16), (0.495, 10), (0.49, 8), (0.485, 6),(0.48, 4), (0.47, 2), (-1., 1)]
# ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

set_random_seed(SEED+NUM_FOLDS-1)

In [ ]:
from torch.utils.data import Dataset

class LitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False, is_target_sampling=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
         
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)    
            self.err_std = torch.tensor(df.standard_error.values, dtype=torch.float32)   
        self.is_target_sampling = is_target_sampling
        self.encoded = tokenizer(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            # 注意这里要手动把return_attention_mask打开
            return_attention_mask=True
#             return_tensors='pt'
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            if self.is_target_sampling:
                err_std = self.err_std[index]
            # 这里对std可以有多种映射，这里先只尝试std=err_std
                
                sampled_target = torch.normal(mean=target, std=RANDOM_FACTOR_OF_TARGET_SAMPLING*err_std, 
                                              size=(1,1)).item()
                sampled_target = torch.tensor([sampled_target], dtype=torch.float32)[0]
                return (input_ids, attention_mask, sampled_target)
            else:
                return (input_ids, attention_mask, target)

In [ ]:
test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
def make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH):

    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

    test_dataset = LitDataset(test_df, tokenizer, inference_only=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             drop_last=False, shuffle=False, num_workers=NUM_WORKERS)

    for index in range(NUM_FOLDS):            
        print(f"\nUsing {model_path}")

        model = LitModel()
        model.load_state_dict(torch.load(f'{input_path}/{model_path}_FOLD{index+1}.pth'))    
        model.to(DEVICE)

        temp_pred = predict(model, test_loader)
        print(temp_pred)
        result_sub = np.zeros(len(temp_pred)) 
        result_sub[:] = temp_pred.flatten()
        result_sub_df = pd.DataFrame(result_sub, columns=['pred'])
        result_sub_df = result_sub_df.to_csv(f'sub_{model_path}_FOLD{index+1}.csv', columns=['pred'])

        del model, temp_pred, result_sub, result_sub_df
        gc.collect()
        torch.cuda.empty_cache()

# Model_1

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average


class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

    
class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)
    
class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)
    
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(abs(self.layer_start), 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), 1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        hidden_state_images = torch.stack(roberta_output.hidden_states)
        
        hidden_state_images = hidden_state_images[self.layer_start:, :, :, :]
        hidden_state_images = hidden_state_images.permute(1, 0, 2, 3)
#         hidden_state_images : size(batch_size * |layer_start| * seq_len * hidden_num)
        context_features = self.conv_neck(hidden_state_images)
        return self.regressor(context_features)

In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0718-a-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0718a"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
# 
make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_2

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

### Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -9
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        last_layer_hidden_states = roberta_output.hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)  

        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0718-b-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0718b"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_3

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

### Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        sequence_output = self.pooler(roberta_output.hidden_states)
        weights = self.attention(sequence_output)
        context_vector = torch.sum(weights * sequence_output, dim=1)  

        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0718-c-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0718c"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"


make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_4

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

### Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -9
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        sequence_output = self.pooler(roberta_output.hidden_states)
        weights = self.attention(sequence_output)
        context_vector = torch.sum(weights * sequence_output, dim=1)  

        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0718-d-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0718d"

In [ ]:
TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_5

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

### Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -9
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(abs(self.layer_start), 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), 1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        hidden_state_images = torch.stack(roberta_output.hidden_states)
        
        hidden_state_images = hidden_state_images[self.layer_start:, :, :, :]
        hidden_state_images = hidden_state_images.permute(1, 0, 2, 3)
#         hidden_state_images : size(batch_size * |layer_start| * seq_len * hidden_num)
        context_features = self.conv_neck(hidden_state_images)
        return self.regressor(context_features)

In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0724-a-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0724a"

In [ ]:
TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_6

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm



class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0713-a-stratified-large-roberta/'
model_path = f"Roberta_large_model_0713a"

In [ ]:
TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_7

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average
    
class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm
    
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0726-a-stratified-large-roberta/'
model_path = f"Roberta_large_model_0726a"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_8

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)



class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -9
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
        

#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        last_layer_hidden_states = roberta_output.hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)  

        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0726-b-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0726b"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_9

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0726-c-stratified-large-roberta/'
model_path = f"Roberta_large_model_0726c"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_10

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 64),
            nn.PReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0727-b-stratified-large-roberta/'
model_path = f"Roberta_large_model_0727b"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_11

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0727-c-stratified-large-roberta/'
model_path = f"Roberta_large_model_0727c"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_12

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.layer_norm = nn.LayerNorm(1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_layer_hidden_states.size()).float()
        sum_embeddings = torch.sum(last_layer_hidden_states * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        norm_mean_embeddings = self.layer_norm(mean_embeddings)
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(norm_mean_embeddings)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0727-f-stratified-large-roberta/'
model_path = f"Roberta_large_model_0727f"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_13

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

# Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -2
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        sequence_output = self.pooler(roberta_output.hidden_states)
        weights = self.attention(sequence_output)
        context_vector = torch.sum(weights * sequence_output, dim=1)  

        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0728-c-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0728c"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_14

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

# Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -3
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        sequence_output = self.pooler(roberta_output.hidden_states)
        weights = self.attention(sequence_output)
        context_vector = torch.sum(weights * sequence_output, dim=1)  

        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0728-d-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0728d"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_15

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

# Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -2
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(abs(self.layer_start), 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), 1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        hidden_state_images = torch.stack(roberta_output.hidden_states)
        
        hidden_state_images = hidden_state_images[self.layer_start:, :, :, :]
        hidden_state_images = hidden_state_images.permute(1, 0, 2, 3)
#         hidden_state_images : size(batch_size * |layer_start| * seq_len * hidden_num)
        context_features = self.conv_neck(hidden_state_images)
        return self.regressor(context_features)



In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0728-e-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0728e"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_16

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        
        
        
        
        
        
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0712-b-stratified-large-roberta/'
model_path = f"Roberta_large_model_0712b"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_17

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 16),
            nn.PReLU(),
            nn.Linear(16, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0729-a-stratified-large-roberta/'
model_path = f"Roberta_large_model_0727b"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_18

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 16),
            nn.PReLU(),
            nn.Linear(16, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0729-b-stratified-large-roberta/'
model_path = f"Roberta_large_model_0727b"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_19

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 64),
            nn.LeakyReLU(0.1),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]
#         print(last_layer_hidden_states.shape)
        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0729-d-stratified-large-roberta/'
model_path = f"Roberta_large_model_0729d"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_20

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

# Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -6
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(abs(self.layer_start), 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.conv_neck = CNNHead(abs(self.layer_start), 1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        hidden_state_images = torch.stack(roberta_output.hidden_states)
        
        hidden_state_images = hidden_state_images[self.layer_start:, :, :, :]
        hidden_state_images = hidden_state_images.permute(1, 0, 2, 3)
#         hidden_state_images : size(batch_size * |layer_start| * seq_len * hidden_num)
        context_features = self.conv_neck(hidden_state_images)
        return self.regressor(context_features)



In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0730-b-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0730b"

In [ ]:
TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_21

In [ ]:


class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.layer_norm = nn.LayerNorm(1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_layer_hidden_states.size()).float()
        sum_embeddings = torch.sum(last_layer_hidden_states * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        norm_mean_embeddings = self.layer_norm(mean_embeddings)
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(norm_mean_embeddings)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0730-c-stratified-large-roberta/'
model_path = f"Roberta_large_model_0730c"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_22

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input

        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.layer_norm = nn.LayerNorm(1024)
        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_layer_hidden_states.size()).float()
        sum_embeddings = torch.sum(last_layer_hidden_states * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        norm_mean_embeddings = self.layer_norm(mean_embeddings)
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(norm_mean_embeddings)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0731-a-stratified-large-roberta/'
model_path = f"Roberta_large_model_0731a"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_23

In [ ]:


class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 16),
            nn.PReLU(),
            nn.Linear(16, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        sequence_output = self.pooler(roberta_output.hidden_states)
        weights = self.attention(sequence_output)
        context_vector = torch.sum(weights * sequence_output, dim=1)  

        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0731-b-stratified-large-roberta/'
model_path = f"Roberta_large_model_0731b"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_24

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        self.layer_start = -8
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=self.layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            self.arcface
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 16),
            nn.PReLU(),
            nn.Linear(16, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)   
        sequence_output = self.pooler(roberta_output.hidden_states)
        weights = self.attention(sequence_output)
        context_vector = torch.sum(weights * sequence_output, dim=1)  

        return self.regressor(context_vector)



In [ ]:
ROBERTA_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

input_path = '../input/0731-e-stratified-large-roberta/'
model_path = f"Roberta_large_model_0731e"

TOKENIZER_PATH = "../input/0712-a-pretrain-large-pipeline/my_roberta_large_pretrained_0711a"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Model_25

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * abs(self.layer_start), dtype=torch.float)
            )

    def forward(self, all_layer_embeddings_input):
#         print('num',self.num_hidden_layers )
        ft_all_layers = all_layer_embeddings_input
        # these teo lines convert 'tuple of tensors' into 'tensor', and then slice on it
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]
#         print('h',all_layer_embedding.shape)
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

class ArcFaceClassifier(nn.Module):
    def __init__(self, emb_size, output_classes):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(emb_size, output_classes))
        nn.init.kaiming_uniform_(self.W)
    def forward(self, x):
        # Step 1:
        x_norm = F.normalize(x)
        W_norm = F.normalize(self.W, dim=0)
        # Step 2:
        return x_norm @ W_norm

class Squeeze(nn.Module):
    def __init__(self, dims=-1):
        super().__init__()
        self.dims = dims
        
    def forward(self, x):
        return x.squeeze(self.dims)

class CNNHead(nn.Module):
    def __init__(self, in_channel, out_num, kernel_size=10):
        super().__init__() 
        self.head = nn.Sequential(
            nn.Conv2d(in_channel, 8, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((64, 256)),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((16, 64)),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=kernel_size),
            nn.AdaptiveMaxPool2d((4, 16)),
            nn.BatchNorm2d(32),
            # batch_size * 32 * 4 * 16
            nn.Flatten(),
            nn.Linear(32 * 4 * 16, out_num)
        )
        
    def forward(self, x):
        return self.head(x)

# Body

class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7,
                       'output_hidden_states':True})   
        
        freeze_embedding = IS_FREEZE_EMBEDDING
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config, )  
        self.roberta.base_model.embeddings.requires_grad_(not freeze_embedding)    
        
        
#         self.config_hidden_size = config.hidden_size
        
        layer_start = 4
        self.pooler = WeightedLayerPooling(
            config.num_hidden_layers, 
            layer_start=layer_start, layer_weights=None
        )
        self.arcface = ArcFaceClassifier(emb_size=512,output_classes=1)
        
        self.attention = nn.Sequential(            
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),            
            nn.Tanh(),                       
            self.arcface
        )        
        self.layer_norm = nn.LayerNorm(ROBERTA_HIDDEN_LEN)
        self.regressor = nn.Sequential(                        
            nn.Linear(ROBERTA_HIDDEN_LEN, 512),
            nn.PReLU(),
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Linear(64, 1)
        )
#         self.fc = nn.Linear(config.hidden_size, 1)


    def forward(self, input_ids, attention_mask):
        
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_layer_hidden_states.size()).float()
        sum_embeddings = torch.sum(last_layer_hidden_states * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        norm_mean_embeddings = self.layer_norm(mean_embeddings)
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(norm_mean_embeddings)



In [ ]:
ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

input_path = '../input/0801-c-stratified-maunish-roberta'
model_path = f"Roberta_maunish_model_0801c"

TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"

make_submission(ROBERTA_PATH, input_path, model_path, TOKENIZER_PATH)

# Submission

In [ ]:
model_path_list = [
    f"Roberta_large_model_0712b",
    f"Roberta_large_model_0713a",
    f"Roberta_maunish_model_0718a",
    f"Roberta_maunish_model_0718b",
    f"Roberta_maunish_model_0718c",
    f"Roberta_maunish_model_0718d",
    f"Roberta_maunish_model_0724a",
    f"Roberta_large_model_0726a",
    f"Roberta_maunish_model_0726b",
    f"Roberta_large_model_0726c",
    f"Roberta_large_model_0727b",
    f"Roberta_large_model_0727c",
    f"Roberta_large_model_0727f",
    f"Roberta_maunish_model_0728c",
    f"Roberta_maunish_model_0728d",
    f"Roberta_maunish_model_0728e",
    f"Roberta_large_model_0727b", # 29a
    f"Roberta_large_model_0727b", # 29b
    f"Roberta_large_model_0729d",
    f"Roberta_maunish_model_0730b",
    f"Roberta_large_model_0730c",
    f"Roberta_large_model_0731a",
    f"Roberta_large_model_0731b",
    f"Roberta_large_model_0731e",
    f"Roberta_maunish_model_0801c",
                  ]

In [ ]:
opt_w

In [ ]:
for model_idx, model_path in enumerate(model_path_list):
    for index in range(NUM_FOLDS):            
        print(f"\nUsing {model_path}")

        sub_i_df = pd.read_csv(f'sub_{model_path}_FOLD{index+1}.csv')
        submission_df.target += sub_i_df.pred * opt_w[model_idx] / NUM_FOLDS
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)